In [ ]:
# leer df_sin_anomalos.csv
import pandas as pd
df_final = pd.read_csv("df_final.csv")

In [ ]:
same = df_final[["SAME-TT2R2_MAS3","TRT"]]

In [ ]:
indice_test = pd.read_csv("indices_test.csv")
indice_train = pd.read_csv("indices_train.csv")

df_test = df_final.iloc[indice_test["0"],:]
df_train = df_final.iloc[indice_train["0"],:]

In [ ]:
SAME_test = same.iloc[indice_test["0"],:]
SAME_train = same.iloc[indice_train["0"],:]

In [ ]:
SAME_X_test= SAME_test.drop(columns=["TRT"])
SAME_y_test = pd.DataFrame(SAME_test["TRT"])

SAME_X_train = SAME_train.drop(columns=["TRT"])
SAME_y_train = pd.DataFrame(SAME_train["TRT"])

In [ ]:
X_test_todos = df_test.drop(columns=["TRT"])
y_test = pd.DataFrame(df_test["TRT"])

X_train_todos = df_train.drop(columns=["TRT"])
y_train = pd.DataFrame(df_train["TRT"])

In [ ]:
X_test_final = X_test_todos[["ARTERIOPATIA_PERIFERICA","ENFERMEDAD_VASCULAR","SAME_MAS_DE_2","N_PASTILLAS_D_INICIO","NEOPLASIA_NO","ADE","MONOCITOS","DISLIPEMIA","AMIODARONA","EDAD","INSUFICIENCIA_CARDIACA","SAME-TT2R2_MAS3","NEUTROFILOS"]]
X_train_final = X_train_todos[["ARTERIOPATIA_PERIFERICA","ENFERMEDAD_VASCULAR","SAME_MAS_DE_2","N_PASTILLAS_D_INICIO","NEOPLASIA_NO","ADE","MONOCITOS","DISLIPEMIA","AMIODARONA","EDAD","INSUFICIENCIA_CARDIACA","SAME-TT2R2_MAS3","NEUTROFILOS"]]

In [ ]:
# hacer una funcion que entrene modelos de regresion mediante gridsearch y los valide con el dataset de validacion y cross validation
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.metrics import classification_report
from scipy.stats import f_oneway
from lifelines.utils import concordance_index
import statsmodels.api as sm
import time
# sequential
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from keras.wrappers.scikit_learn import KerasRegressor
from keras import regularizers
# linear regression
from sklearn.linear_model import LinearRegression

# no mostrar warnings
import warnings

# r2 ajustado
def r2_adj(r2, n, p):
    r2_adj = 1 - ((1 - r2) * (n - 1) / (n - p - 1))
    return r2_adj

def train_model_gridsearch(model, X_train, X_val, y_train, y_val, params,DF):
    warnings.filterwarnings('ignore')
    grid = GridSearchCV(model, params, cv=5, scoring=["neg_root_mean_squared_error",'neg_mean_absolute_error', 'neg_mean_squared_error', 'r2'], refit='r2')
    grid.fit(X_train, y_train)
    print("Best params: ", grid.best_params_)
    print("Best score: ", grid.best_score_)
    start = time.time()
    y_pred = grid.predict(X_val)
    end = time.time()
    print("Time: ", end - start)
    print()
    print("MAE: ", mean_absolute_error(y_val, y_pred))
    # desviacion estandar de MAE
    print("MAE std: ", np.std(mean_absolute_error(y_val, y_pred)))
    print("MSE: ", mean_squared_error(y_val, y_pred))
    print("RMSE: ", np.sqrt(mean_squared_error(y_val, y_pred)))
    print("R2: ", r2_score(y_val, y_pred))
    print("R2 ajustado: ", r2_adj(r2_score(y_val, y_pred), len(y_val), len(X_val.columns)))
    print()

    # residuos
    residuos = y_val - y_pred

    # crear un dataframe donde se vayan almacenando los resultados de cada modelo
    DF = DF.append({"Modelo": str(model),"R2 ajustado":r2_adj(r2_score(y_val, y_pred), len(y_val), len(X_val.columns)),"R2 ajustado std": np.std(r2_adj(r2_score(y_val, y_pred), len(y_val), len(X_val.columns))),"MAE": mean_absolute_error(y_val, y_pred),"MAE_std":np.sqrt(mean_squared_error(y_val, y_pred)), "MSE": mean_squared_error(y_val, y_pred),"MSE std": np.std(mean_squared_error(y_val, y_pred)) , "RMSE": np.sqrt(mean_squared_error(y_val, y_pred)),"RMSE std": np.std(np.sqrt(mean_squared_error(y_val, y_pred))), "R2": r2_score(y_val, y_pred),"R2 std": np.std(r2_score(y_val, y_pred)), "Time": end - start,"Residuos":residuos}, ignore_index=True)

    # grafico de real vs pred
    plt.scatter(y_val, y_pred)
    plt.title("Real vs Predicted")
    plt.xlabel("Real")
    plt.ylabel("Predicted")
    # same axis 0-100
    plt.plot([0, 100], [0, 100], 'k-', color = 'r')
    plt.show()

    # grafico de distribucion de residuos
    sns.distplot(y_val - y_pred)
    plt.title("Distribution of Residuals")
    plt.xlabel("Residuals")
    plt.ylabel("Frequency")
    plt.show()

    return DF

In [ ]:
SIGNIFICATIVAS = pd.DataFrame(columns=["Modelo","R2 ajustado","R2 ajustado std", "MAE", "MAE_std","MSE","MSE std", "RMSE","RMSE std", "R2","R2 std" "Time","Residuos"])
TODAS = pd.DataFrame(columns=["Modelo","R2 ajustado","R2 ajustado std", "MAE", "MAE_std","MSE","MSE std", "RMSE","RMSE std", "R2","R2 std" "Time","Residuos"])

In [ ]:
SAME = pd.DataFrame(columns=["Modelo","R2 ajustado","R2 ajustado std", "MAE", "MAE_std","MSE","MSE std", "RMSE","RMSE std", "R2","R2 std" "Time","Residuos"])

In [ ]:
# lista con los modelos a probar (linear regression, svr, neural network, decisiom tree and regresion polinomica) y sus parametros
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
# voting regressor
from sklearn.ensemble import VotingRegressor
# sequential
# ada boost regressor
from sklearn.ensemble import AdaBoostRegressor


models = [SVR(), DecisionTreeRegressor(), KNeighborsRegressor(),MLPRegressor(),LinearRegression(),AdaBoostRegressor()]
params = [{"kernel": ["linear", "poly", "rbf", "sigmoid"], "C": [0.1, 1, 10, 100, 1000], "gamma": ["scale", "auto"]},
            {"criterion": ["mse", "friedman_mse", "mae"], "splitter": ["best", "random"], "max_depth": [None, 1,2, 3,4, 5,6, 7, 8, 9,10]},
            {"n_neighbors": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], "weights": ["uniform", "distance"], "algorithm": ["auto", "ball_tree", "kd_tree", "brute"]},
            {"hidden_layer_sizes": [(50,50,50), (50,100,50), (100,)], "activation": ["tanh", "relu"], "solver": ["sgd", "adam"], "learning_rate": ["constant", "invscaling", "adaptive"]},
            {},
            {"n_estimators": [50, 100, 150, 200, 250, 300, 350, 400, 450, 500], "learning_rate": [0.001, 0.01, 0.1, 1, 10, 100, 1000], "loss": ["linear", "square", "exponential"]}]

# entrenar los modelos con el dataset de entrenamiento y validarlos con el dataset de validacion
for i in range(len(models)):
    print("Model: ", models[i])
    TODAS = train_model_gridsearch(models[i], X_train_todos, X_test_todos, y_train["TRT"], y_test["TRT"], params[i],TODAS)
    SIGNIFICATIVAS = train_model_gridsearch(models[i], X_train_final, X_test_final, y_train["TRT"], y_test["TRT"], params[i],SIGNIFICATIVAS)
    SAME = train_model_gridsearch(models[i], SAME_X_train, SAME_X_test, SAME_y_train["TRT"], SAME_y_test["TRT"], params[i],SAME)
    print("")

In [ ]:
# VOTING de linear regression, knn y mlp
models = [("lr", LinearRegression()), ("knn", KNeighborsRegressor(n_neighbors=10)), ("ada", AdaBoostRegressor(learning_rate=0.01,n_estimators=150))]
voting = VotingRegressor(estimators=models)
params = {"weights": [[1,1,1], [1,2,1], [1,1,2], [1,2,2], [2,1,1], [2,2,1], [2,1,2], [2,2,2]]}
print("Model: ", voting)
TODAS = train_model_gridsearch(voting, X_train_todos, X_test_todos, y_train["TRT"], y_test["TRT"], params,TODAS)
SIGNIFICATIVAS = train_model_gridsearch(voting, X_train_final, X_test_final, y_train["TRT"], y_test["TRT"], params,SIGNIFICATIVAS)
SAME = train_model_gridsearch(voting, SAME_X_train, SAME_X_test, SAME_y_train["TRT"], SAME_y_test["TRT"], params,SAME)
print("")

In [ ]:
SIGNIFICATIVAS

In [ ]:
TODAS

In [ ]:
SAME

In [ ]:
for df in [TODAS, SIGNIFICATIVAS]:
    for i in range(len(TODAS)):
        df["Residuos"][i] = df["Residuos"][i].to_list()

In [ ]:
for i in range(len(SAME)):
    SAME["Residuos"][i] = SAME["Residuos"][i].to_list()

In [ ]:
modelos = ["SVR", "DT", "KNN", "MLP", "LR", "AdaBoost", "Voting"]

In [ ]:
plt.figure(figsize=(20,10))
sns.boxplot(data=SIGNIFICATIVAS["Residuos"])
plt.gca().set_xticklabels(modelos,rotation=45, ha='right')
# cambiar el tamaño de la letra
plt.rc('xtick', labelsize=12)
# cambiar los labels de los ejes
# cambiar tamaño ejes
plt.rc('axes', labelsize=12)

plt.title("Boxplot de Residuos")
plt.xlabel("Modelo")
plt.ylabel("Residuos")
plt.show()

In [ ]:
# sacar la mediana y el rango intercuartilico de cada modelo
res = {}
for df in [TODAS,SIGNIFICATIVAS]:
    for i in range(len(df)):
        print(df["Modelo"][i])
        print("Mediana: ", np.median(df["Residuos"][i]))
        print("STD: ",np.std(df["Residuos"][i]))
        print("IQR: ", np.percentile(df["Residuos"][i], 75) - np.percentile(df["Residuos"][i], 25))

In [ ]:
# normalidad de los residuos
from scipy.stats import shapiro
for df in [TODAS,SIGNIFICATIVAS]:
    for i in range(len(df)):
        print(df["Modelo"][i])
        print(shapiro(df["Residuos"][i]))

In [ ]:
# homogeneidad de varianzas para todos los modelos
from scipy.stats import levene

levene(df["Residuos"][0], df["Residuos"][1], df["Residuos"][2], df["Residuos"][3], df["Residuos"][4], df["Residuos"][5], df["Residuos"][6])


In [ ]:
# realizar un test de anova para ver si hay diferencias significativas entre los modelos
# sacar los residuos de cada modelo
residuos = []
for i in range(len(SIGNIFICATIVAS)):
    residuos.append(SIGNIFICATIVAS["Residuos"][i])

# test de anova
f_oneway(*residuos)

In [ ]:
# boxplot de los residuos de cada modelo
plt.figure(figsize=(20,10))
sns.boxplot(data=SIGNIFICATIVAS["Residuos"])
plt.gca().set_xticklabels(modelos, rotation=45, ha='right')
plt.title("Boxplot of Residuals")
plt.xlabel("Model")
plt.ylabel("Residuals")
plt.show()

In [ ]:
# examinar el valor atipico de cada modelo
for i in range(len(SIGNIFICATIVAS)):
    print(modelos[i])
    print(sorted(SIGNIFICATIVAS["Residuos"][i]))
    print()

In [ ]:
# sacar la fila del valor atipico de cada modelo
for i in range(len(SIGNIFICATIVAS)):
    print(modelos[i])
    print(SIGNIFICATIVAS["Residuos"][i].index(max(SIGNIFICATIVAS["Residuos"][i])))
    print()

In [ ]:
# analizar la fila 154
df_final.iloc[154]